# Data Cleaning - comments_withLabels.tsv

Description:
- cleaned comments_withLabels ready for Naive Bayes and SVM

## Import Libraries

### Main Libraries

In [1]:
import pandas as pd
import numpy as np
import re

### NLP Libraries

In [2]:
# NLTK
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# Gensim
from gensim.parsing.porter import PorterStemmer

## Load Dataset

In [3]:
# Load dataset
df = pd.read_csv('C:/Users/cherryb/Desktop/Personal Projects/Datasets/Telus - Fintech/comments_withLabels.tsv', sep='\t')
# Inspect df
df.head(3)

,Unnamed: 0,position,post_id,post_by,post_text,post_published,comment_id,comment_by,is_reply,comment_message,comment_published,comment_like_count,attachment_type,attachment_url,Sentiments
0,0,20_0,155027942462_10157280442467463,f0a137b6d4a4b4e94cc04bea6ac43d203c12d679,Let us help keep you in control of your money ...,2019-05-31T13:42:05+0000,10157280442467463_10157324996792463,da39a3ee5e6b4b0d3255bfef95601890afd80709,1,Why?,2019-06-18T00:08:26+0000,0,NaN,NaN,Neutral
1,1,20_1,155027942462_10157280442467463,f0a137b6d4a4b4e94cc04bea6ac43d203c12d679,Let us help keep you in control of your money ...,2019-05-31T13:42:05+0000,10157280442467463_10157291740937463,da39a3ee5e6b4b0d3255bfef95601890afd80709,0,The worst card ever it’s lawsuit time,2019-06-04T19:53:28+0000,6,NaN,NaN,Negative
2,2,32_0,155027942462_10157236349992463,f0a137b6d4a4b4e94cc04bea6ac43d203c12d679,Rise & Grind ☀️ / Plan today for your success ...,2019-05-13T15:41:07+0000,10157236349992463_10157237648947463,da39a3ee5e6b4b0d3255bfef95601890afd80709,0,I think some needs to hear this. Over the year...,2019-05-14T03:28:34+0000,0,NaN,NaN,Neutral


## Data Pre-processing

In [4]:
# Drop unnecessary columns
drop_columns = ['Unnamed: 0', 'position', 'post_id', 'post_by', 'post_text', 
                'post_published', 'comment_id', 'attachment_url']
df = df.drop(drop_columns, axis=1)

In [5]:
# Check for null values
df.isnull().sum()

comment_by               0
is_reply                 0
comment_message          0
comment_published        0
comment_like_count       0
attachment_type       3217
Sentiments               0
dtype: int64

In [6]:
# Delete comments that are posted by pageowner
df = df.drop(df[df.comment_by == 'pageowner'].index)

# Drop comment_by column
df = df.drop(['comment_by'], axis=1)

In [7]:
def delete_url(text):
    '''
    To delete rows that looks like a URL
    (https, http, www)
    '''
    return re.sub(r'''(?i)\b((?:http[s]?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', 'None', text)

In [8]:
# To delete and drop URL comments
df['comment_message'] = df['comment_message'].apply(delete_url)
df = df.drop(df[df.comment_message == 'None'].index)
df.head()

,is_reply,comment_message,comment_published,comment_like_count,attachment_type,Sentiments
0,1,Why?,2019-06-18T00:08:26+0000,0,NaN,Neutral
1,0,The worst card ever it’s lawsuit time,2019-06-04T19:53:28+0000,6,NaN,Negative
2,0,I think some needs to hear this. Over the year...,2019-05-14T03:28:34+0000,0,NaN,Neutral
3,0,"How long does it take to get refunded money, P...",2019-04-20T20:44:07+0000,1,NaN,Negative
4,0,REALLY? Recipes From Heaven,2019-03-15T22:37:37+0000,0,NaN,Neutral


In [9]:
# Tokenize the posts
df['comment_message'] = df['comment_message'].apply(lambda list_words: word_tokenize(list_words))

In [10]:
def remove_nonalpha(text):
    '''
    Removing non-alpha characters
    '''
    return re.sub('[^a-zA-Z]', '', text)

In [11]:
# Run the function to remove non-letter characters
df['comment_message'] = df['comment_message'].apply(lambda list_words: [remove_nonalpha(word) for word in list_words])

In [12]:
# Remove empty strings
df['comment_message'] = df['comment_message'].apply(lambda list_words: list(filter(None, list_words)))

In [13]:
# Convert all letters to lowercase
df['comment_message'] = df['comment_message'].apply(lambda list_words: [word.lower() for word in list_words])

In [14]:
# Remove stopwords
stop_words = sorted(stopwords.words('english'))
stop_words.append('http')
df['comment_message'] = df['comment_message'].apply(lambda list_words: [word for word in list_words if not word in stop_words])

In [15]:
# Sort words by grouping inflected forms of the same word
lem = WordNetLemmatizer()
df['comment_message'] = df['comment_message'].apply(lambda list_words: [lem.lemmatize(word) for word in list_words])

In [16]:
# Eliminate affixes using gensim's PporterStemmer
p = PorterStemmer()
df['comment_message'] = df['comment_message'].apply(lambda list_words: [p.stem(word) for word in list_words])

In [17]:
df.head()

,is_reply,comment_message,comment_published,comment_like_count,attachment_type,Sentiments
0,1,[],2019-06-18T00:08:26+0000,0,NaN,Neutral
1,0,"[worst, card, ever, lawsuit, time]",2019-06-04T19:53:28+0000,6,NaN,Negative
2,0,"[think, need, hear, year, think, bitcoin, impa...",2019-05-14T03:28:34+0000,0,NaN,Neutral
3,0,"[long, take, get, refund, monei, paypal, state...",2019-04-20T20:44:07+0000,1,NaN,Negative
4,0,"[realli, recip, heaven]",2019-03-15T22:37:37+0000,0,NaN,Neutral


## Save as comments_cleaned.tsv

In [18]:
df.to_csv('C:/Users/cherryb/Desktop/Personal Projects/Datasets/Telus - Fintech/cleaned/commentsCleaned_NV-SVM.tsv', sep='\t')